In [25]:
# 1. 加载必要的库
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, Trainer
from peft import LoraConfig, get_peft_model

In [26]:
# 2. 加载数据集
from datasets import load_dataset

data_files = {
    "train": ["./Medical task training dataset/train_zh_0.json", 
              "./Medical task training dataset/train_en_1.json"], 
    "validation": ["./Medical task training dataset/valid_zh_0.json", 
                   "./Medical task training dataset/valid_en_1.json"],
    "test": ["./Medical task training dataset/test_zh_0.json", 
             "./Medical task training dataset/test_en_1.json"]
}

dataset = load_dataset('json', data_files=data_files)

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
# 3. 加载模型和分词器
model_name = "qwen/Qwen2-1.5B-Instruct"  # ModelScope 的模型 ID

# int4 量化配置 (保持不变)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,  # 应用量化配置
    torch_dtype=torch.bfloat16,  # 使用 float16 精度
    trust_remote_code=True,
)


2024-12-25 02:37:15,750 - modelscope - INFO - Creating symbolic link [C:\Users\asset\.cache\modelscope\hub\qwen/Qwen2-1.5B-Instruct].
2024-12-25 02:37:15,750 - modelscope - WARNING - Failed to create symbolic link C:\Users\asset\.cache\modelscope\hub\qwen/Qwen2-1.5B-Instruct.


2024-12-25 02:37:18,297 - modelscope - INFO - Creating symbolic link [C:\Users\asset\.cache\modelscope\hub\qwen/Qwen2-1.5B-Instruct].
2024-12-25 02:37:18,297 - modelscope - WARNING - Failed to create symbolic link C:\Users\asset\.cache\modelscope\hub\qwen/Qwen2-1.5B-Instruct.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,)
def preprocess_function(examples):
    # 使用 tokenizer 对 instruction 和 input 进行分词
    inputs = tokenizer(
        examples["instruction"],
        examples["input"],
        truncation="longest_first",  # 优先截断较长的序列
        max_length=2048,
        padding="max_length",
        return_tensors="pt",
    )

    # 添加特殊标记
    inputs["input_ids"] = tokenizer.build_inputs_with_special_tokens(inputs["input_ids"]) 
    inputs["attention_mask"] = tokenizer.build_inputs_with_special_tokens(inputs["attention_mask"])

    # 根据模型要求添加 labels 字段
    inputs["labels"] = inputs["input_ids"].clone() 

    return inputs

dataset = dataset.map(preprocess_function, batched=True, num_proc=4)  # 使用 4 个进程

print(dataset["train"][0])  # 查看第一个训练样本

Map (num_proc=4):   0%|          | 0/2066589 [00:01<?, ? examples/s]


NameError: name 'tokenizer' is not defined